In [1]:
# Learn to use: ImageFolder
# Scheduler
# Transfer Learning

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])
learningRate = 0.001

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}


In [8]:
#import data
data_dir = './data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

['ants', 'bees']


In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [15]:
# option 1: retrain the entire fully connected neural network
# import pretrain model
model = models.resnet18(pretrained=True)
# change last layer in fully connected network
num_ftrs = model.fc.in_features

#now new model only have 2 classes (ant and bee)
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learningRate)

# scheduler
# every 7 epoch, our learning rate *= gamma
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)


Epoch 0/19
----------
train Loss: 0.6228 Acc: 0.6148
val Loss: 0.4168 Acc: 0.8497

Epoch 1/19
----------
train Loss: 0.4959 Acc: 0.7664
val Loss: 0.3324 Acc: 0.9085

Epoch 2/19
----------
train Loss: 0.4666 Acc: 0.8156
val Loss: 0.2786 Acc: 0.9216

Epoch 3/19
----------
train Loss: 0.3838 Acc: 0.8279
val Loss: 0.2536 Acc: 0.9216

Epoch 4/19
----------
train Loss: 0.3840 Acc: 0.8320
val Loss: 0.2360 Acc: 0.9281

Epoch 5/19
----------
train Loss: 0.3970 Acc: 0.8074
val Loss: 0.2432 Acc: 0.9281

Epoch 6/19
----------
train Loss: 0.3796 Acc: 0.8238
val Loss: 0.2276 Acc: 0.9216

Epoch 7/19
----------
train Loss: 0.3600 Acc: 0.8484
val Loss: 0.2176 Acc: 0.9281

Epoch 8/19
----------
train Loss: 0.3271 Acc: 0.8525
val Loss: 0.2058 Acc: 0.9412

Epoch 9/19
----------
train Loss: 0.3036 Acc: 0.8648
val Loss: 0.2220 Acc: 0.9216

Epoch 10/19
----------
train Loss: 0.2749 Acc: 0.9057
val Loss: 0.2069 Acc: 0.9346

Epoch 11/19
----------
train Loss: 0.3177 Acc: 0.8607
val Loss: 0.2118 Acc: 0.9346

Ep

In [ ]:
# second option, reuse all layer from begining and only retrain the last layer
# import pretrain model
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features

#now new model only have 2 classes (ant and bee)
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learningRate)

# scheduler
# every 7 epoch, our learning rate *= gamma
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)
